# An air conditioner problem
A stage-wise independent finite discrete MSLP.    
The data process is stage-wise independent and on the RHS.    
It was originally from
http://www.optimization-online.org/DB_FILE/2017/12/6388.pdf    
Verified optimal value by extensive solver is 68200.    
SDDP solver also obtains the same optimal value.

## Formulation
\begin{align*}
    \min~ & \sum_{t=1}^T x_t + 3 w_t + 0.5 y_t\\
    s.t.~ & \textrm{for}~ t = 1,\dots,T\\
         & x_t \leq 2\\
         & x_t + w_t + y_{t-1} - y_t = d_t\\
         & y_0 = 0, d_1 = 100, 
             d_2,\dots,d_T\overset{i.i.d}{\sim}
                \begin{cases}
                    100, \textrm{w.p.}~ 0.4\\
                    300, \textrm{w.p.}~ 0.6
                \end{cases}
\end{align*}

In [4]:
from msppy.msp import MSIP
from msppy.solver import SDDiP,Extensive
import gurobipy
T = 3
A = [100,300]
airConditioners = MSIP(T=T, bound=0)
for t in range(T):
    m = airConditioners[t]
    storage_now, storage_past = m.addStateVar(vtype='I', obj=50)
    produce = m.addVar(ub=200, vtype='I', obj=100)
    overtime = m.addVar(vtype='I', obj=300)
    m.update()
    if t == 0:
        m.addConstr(produce + overtime - storage_now == 100)
    else:
        m.addConstr(
            storage_past + produce + overtime - storage_now == 0, 
            uncertainty={'rhs': A}
        )
        m.set_probability([0.4,0.6])

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [6]:
Extensive(airConditioners).solve(outputFlag=0)
SDDiP(airConditioners).solve(cuts=['B'], max_iterations=10)

----------------------------------------------------------------
                   SDDP Solver, Lingquan Ding                   
----------------------------------------------------------------
   Iteration               Bound               Value        Time
----------------------------------------------------------------
           1        68200.000000        95000.000000    0.021094
           2        68200.000000        95000.000000    0.018762
           3        68200.000000        55000.000000    0.032083
           4        68200.000000        95000.000000    0.015700
           5        68200.000000        95000.000000    0.013577
           6        68200.000000        60000.000000    0.009194
           7        68200.000000        55000.000000    0.014426
           8        68200.000000        60000.000000    0.013763
           9        68200.000000        95000.000000    0.008099
          10        68200.000000        60000.000000    0.006122
-------------------------